##### 第一模块(导入包模块：必须运行）

In [1]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import time
#time

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
# tk文件导入模块

##### 第二模块(文件处理模块：必须运行）

In [2]:
def import_table_freq(root):
    # file_name = input("请输入已获得的词频表的名称(不用加后缀.xlsx):")
    # file_name_xlsx = file_name + ".xlsx"
    # table = pd.read_excel(file_name_xlsx)
    # return table
    ##### 
    """
    return table:选中的表格
    """
    try:
        print("-----------------------------------------")
        print("*********请选择已获得的词频表的名称*********")
        print("-----------------------------------------")
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")

def import_table_corpus(root):
    # file_name = input("请输入要分析的excel表的名称(不用加后缀.xlsx):")
    # file_name_xlsx = file_name + ".xlsx"
    # table = pd.read_excel(file_name_xlsx)
    # return table
    try:
        print("-----------------------------------------")
        print("*********请选择已获得的全文表的名称*********")
        print("-----------------------------------------")
        root.filename =  filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            return table
    except Exception as e:
        print("导入错误")

def remote_select():
    print("以下文件可以调用，需要分析哪个文件？")
    print("-------------------------------------")
    index = 0
    list_item_temp = []
    for item in os.listdir():
        if ".xlsx" in item:
            index += 1
            print("[" + str(index) + "] " + item)
            list_item_temp.append(item)
    try:
        bash_pos = "/Users/dfuser/Desktop/目标文书目录/"
        file_code = int(input("需要导入哪个文件？(输入[]中的序号)"))
        final_pos = bash_pos + str(list_item_temp[file_code-1])
        table_ = pd.read_excel(final_pos)
        return table_
        print("表格导入成功，以下是表格预览")
        print("----------------------------")
    except Exception as e:
        print("导入错误")
        
def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该最大共现表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
        else:
            pass
    except Exception as e:
        pass

#### 程序主模块

In [3]:
## -*- coding:utf-8 -*-
## 词语共现模块
## 将第一步所得到的词频表合并 
## 共现第一步，找每篇文章里存在多少词
def find_word_in_articles(corpus,word_table):
    """
    word_table:高频词列表，此处设定为：table_topwords["词语"]
    corpus:文档文集，此处设定为：corpus[0:10],只取前十个做范例
    """
    list_vector = []
    list_matrix = []
    for item in word_table:
        for word_list in [sentence for sentence in corpus]:
            if item in str(word_list):
                list_vector.append(1)
            else:
                list_vector.append(0)
        list_matrix.append(list_vector)
        list_vector = []
    return list_matrix
    
## 共现第二步，找词语共现情况
def find_coappear(word1,word2,table_matrix):
    """
    Params:
    word_table:高频词列表，此处设定为：table_topwords["词语"]
    corpus:文档文集，此处设定为：corpus[0:10],只取前十个做范例
    """
    coappear = 0
    for item in zip(table_matrix.T[word1],table_matrix.T[word2]):
        if item == (1,1):
            coappear += 1
    return coappear

def get_list_matrix(table_m):
    list_matrix = []
    list_vector = []
    table_matrix = table_m
    np.shape(table_matrix)
    for word1 in table_matrix.index:
        list_vector = []
        for word2 in table_matrix.index:
            coappear = find_coappear(word1,word2,table_matrix)
            list_vector.append(coappear)
            list_matrix.append(list_vector)
    return list_matrix

def get_table_matrix():
    flag_input = int(input("你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)"))
    if flag_input == 1:
        root = Tk() # 实例化TKinter窗口
        root.withdraw() # 隐藏TKinter窗口
        
        table_ready_to_eat_1 = import_table_freq(root) #导入词频的表

        print("----------------------------------------------")
        print("******************词频表格已经导入******************")
        print("----------------------------------------------")

        table_ready_to_eat_2 = import_table_corpus(root) #导入全文的表

        print("----------------------------------------------")
        print("******************文书表格已经导入******************")
        print("----------------------------------------------")
        
    elif flag_input == 2:
        table_ready_to_eat_1 = remote_select()
        
        print("----------------------------------------------")
        print("******************词频表格已经导入******************")
        print("----------------------------------------------")
        table_ready_to_eat_2 = remote_select()
        
        print("----------------------------------------------")
        print("******************文书表格已经导入******************")
        print("----------------------------------------------")
    else:
        logging("加载错误")
        
    print("表格中所有的列:",table_ready_to_eat_2.columns)
    selected_column = input("请问需要分析哪一列的关键词词频？(输入关键词部分):")
    
    table_ready_to_eat_1.index = table_ready_to_eat_1["word"]
    table_ready_to_eat_1 = table_ready_to_eat_1[0:100] # 取前100个词的共现就可以了，后面词的共现无意义。
    corpus = table_ready_to_eat_2[selected_column] #此处要留出选择模块的口子。
    table_topwords = table_ready_to_eat_1.drop("word",axis = 1)
    table_topwords["词语"] = table_topwords.index
    table_topwords.index = range(len(table_topwords))
    table_topwords.columns = ["频率","词语"]
    
    list_matrix_pre = find_word_in_articles(corpus,table_topwords["词语"]) #find_word_in_articles
    table_matrix_final = pd.DataFrame(list_matrix_pre)
    table_matrix_final.index = table_topwords["词语"]
    return (table_matrix_final,table_topwords)

def get_max_coappear(list_matrix,table_topwords):
    dimension = np.shape(list_matrix)[1]
    table_coappear = pd.DataFrame(list_matrix)[0::dimension]
    table_coappear.index = table_topwords[0:dimension]["词语"]
    table_coappear.columns = table_topwords[0:dimension]["词语"]
    table_coappear["词语"] = table_coappear.index
    table_co = table_coappear
    table_co.index = table_co["词语"]
    list_save_tuple = []
    for item in table_co.index:
        for index,value in zip(table_co[item].index,table_co[item]):
            list_save_tuple.append((item,index,value))
    table_co2 = pd.DataFrame(list_save_tuple)
    table_co2.columns = ["word1","word2","weight"]
    list_unrepeated = []
    for i,j,q in zip(table_co2["word1"],table_co2["word2"],table_co2["weight"]):
        if i != j:
            list_unrepeated.append((i,j,q))
    table_unrepeated = pd.DataFrame(list_unrepeated)
    table_unrepeated.columns = ["word1","word2","weight"]
    table_unrepeated_s = table_unrepeated.sort_values("weight",ascending = False)
    return table_unrepeated_s

#### 控制部分

In [6]:
if __name__ == '__main__': 
    table_matrix_exe = get_table_matrix()
    table_matrix_intermediate = table_matrix_exe[0]
    list_matrix = get_list_matrix(table_matrix_intermediate)
    max_coappear_result = get_max_coappear(list_matrix,table_matrix_exe[1])
    save_model(max_coappear_result)

你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)2
以下文件可以调用，需要分析哪个文件？
-------------------------------------
[1] 夫妻债务 - 原表.xlsx
[2] wordfreqs.xlsx
[3] .~wordfreqs.xlsx
[4] ex1.xlsx
[5] 校园暴力 - 结构化重构.xlsx
[6] 校园暴力事件 - 原表.xlsx
[7] 正当防卫 - 原表.xlsx
[8] 夫妻债务.xlsx
[9] .~zhaiwu_regex_filter.xlsx.xlsx
[10] 职业打假人 - 原表.xlsx
需要导入哪个文件？(输入[]中的序号)2
----------------------------------------------
******************词频表格已经导入******************
----------------------------------------------
以下文件可以调用，需要分析哪个文件？
-------------------------------------
[1] 夫妻债务 - 原表.xlsx
[2] wordfreqs.xlsx
[3] .~wordfreqs.xlsx
[4] ex1.xlsx
[5] 校园暴力 - 结构化重构.xlsx
[6] 校园暴力事件 - 原表.xlsx
[7] 正当防卫 - 原表.xlsx
[8] 夫妻债务.xlsx
[9] .~zhaiwu_regex_filter.xlsx.xlsx
[10] 职业打假人 - 原表.xlsx
需要导入哪个文件？(输入[]中的序号)1
----------------------------------------------
******************文书表格已经导入******************
----------------------------------------------
表格中所有的列: Index(['标题', '案号', '案件类型', '庭审程序', '案由', '文书类型', '法院', '判决日期', '原告', '被告',
       '第三人', '法官', '审判长', '审判员', '书记员'